In [2]:
import os
import sys
import time
import json
import logging
import warnings
import traceback
import numpy as np
import pandas as pd

from fmlc.baseclasses import eFMU

In [3]:
import matplotlib.pyplot as plt

# get root
try:
    root = os.path.dirname(os.path.abspath(__file__))
except:
    root = os.getcwd()
    
# afc modules
sys.path.append(os.path.dirname(root))
from afc.radiance.configs import get_config
from afc.radiance.forecast import Forecast
from afc.utility.weather import read_tmy3

# doper modules
from doper import DOPER, get_solver, resample_variable_ts, standard_report
from doper.computetariff import compute_periods
from doper.data.tariff import get_e19_2020_tariff
from doper.models.basemodel import default_output_list


In [28]:
from afc.optModel import control_model
from afc.defaultConfig import default_parameter
from afc.defaultConfig import ft2_to_m2
from afc.utility.plotting import plot_standard1

In [29]:
import json

with open('current_config.json', 'r') as f:
    config = json.load(f)

In [30]:
config

{'system_id': 'Test-AAA',
 'location_state': 'CA',
 'location_city': 'Berkeley',
 'location_orientation': '180',
 'room_width': '10',
 'room_height': '11',
 'room_depth': '15',
 'occupant_1_location': '2',
 'occupant_1_direction': '1',
 'occupant_2_location': '1',
 'occupant_2_direction': '2',
 'occupant_3_location': '5',
 'occupant_3_direction': '1',
 'occupant_brightness': '100',
 'occupant_glare': '100',
 'window_width': '4.5',
 'window_height': '8.5',
 'window_sill': '0.5',
 'window_count': '2',
 'system_type': 'EC-Sage',
 'system_zones': '3',
 'system_light': 'FLU',
 'system_lux': '400',
 'system_cooling': 'el',
 'system_cooling_eff': '3.5',
 'system_heating': 'el',
 'system_heating_eff': '0.95',
 'location_latitude': '37.85',
 'location_longitude': '-122.24',
 'interface_status': 'Updated Configuration.'}

In [37]:
def read_config(config):
    parameter = default_parameter(precompute_radiance=False)
    parameter['radiance']['location']['longitude'] = float(config['location_longitude'])
    parameter['radiance']['location']['latitude'] = float(config['location_latitude'])
    parameter['radiance']['location']['orient'] = int(config['location_orientation'])
    window_area = ft2_to_m2((float(config['window_height'])-float(config['window_sill'])) * float(config['window_width']) * float(config['window_count']))
    parameter['facade']['window_area'] = window_area
    parameter['radiance']['dimensions']['width'] = float(config['room_width'])
    parameter['radiance']['dimensions']['depth'] = float(config['room_depth'])
    parameter['radiance']['dimensions']['height'] = float(config['room_height'])
    facade_area = ft2_to_m2(float(config['room_width']) * float(config['room_height']))
    wwr = window_area/facade_area
    parameter['radiance']['wwr'] = round(wwr, 1)
    parameter['zone']['heating_efficiency'] = float(config['system_heating_eff'])
    parameter['zone']['cooling_efficiency'] = float(config['system_cooling_eff'])
    parameter['zone']['param']['type'] = config['system_type']
    return parameter

In [38]:
test = read_config(config)

In [40]:
test

{'system': {'pv': True,
  'battery': False,
  'genset': False,
  'load_control': False,
  'external_gen': False,
  'reg_bidding': False,
  'reg_response': False,
  'hvac_control': True},
 'fuels': [{'name': 'ng',
   'unit': 'therms',
   'rate': 3.93,
   'conversion': 30.77,
   'co2': 5.3,
   'reserves': 0},
  {'name': 'diesel',
   'unit': 'gallons',
   'rate': 3.4,
   'conversion': 35.75,
   'co2': 10.18,
   'reserves': 100}],
 'tariff': {'energy': {0: 0.08671, 1: 0.11613, 2: 0.16055},
  'demand': {0: 0, 1: 5.4, 2: 19.65},
  'demand_coincident': 17.74,
  'export': {0: 0.01}},
 'site': {'customer': 'Commercial',
  'regulation': False,
  'regulation_min': None,
  'regulation_all': False,
  'regulation_symmetric': False,
  'regulation_xor_building': False,
  'regulation_xor': False,
  'regulation_reserved': False,
  'regulation_reserved_battery': False,
  'regulation_reserved_variable_battery': False,
  'import_max': 1000000000.0,
  'export_max': 1000000000.0,
  'demand_periods_prev': {0: